# Feature Extraction using the PyRadiomics toolbox in Python

### Imports

In [127]:
import os  # needed navigate the system to get the input data
import pandas as pd
from pathlib import Path


import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

### Load the label data for each sample

In [153]:
# Load the labels
dataset_folder = Path(os.getcwd()+'/dataset')
label = pd.read_excel(dataset_folder/Path('Pretreat-MetsToBrain-Masks_clin_20230918.xlsx'),sheet_name='Data')

# Select and process the labels
# label = label[['BraTS_MET_ID', 'Death']].dropna(axis=0, subset=['Death'])
label = label.dropna(axis=0, subset=['Death'])
label.Death = label.Death.astype(int)

### Instantiating the extractor

##### Important step to define the parms file version

In [145]:
# params_file_name = 'default_params'
params_file_name = 'params_1'

image_types = ('original', 'log-sigma', 'wavelet')

print(f"Setup for parameters: {params_file_name} and image types: {image_types}")

Setup for parameters: params_1 and image types: ('original', 'log-sigma', 'wavelet')


In [146]:
# Location of the parameter file
paramPath = os.path.join('params', params_file_name+'.yaml')
print('Parameter file, absolute path:', os.path.abspath(paramPath))

# Instantiate the extractor with parameters file
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)

print('Extraction parameters:\n\t', extractor.settings)
print('Enabled filters:\n\t', extractor.enabledImagetypes)
print('Enabled features:\n\t', extractor.enabledFeatures)

Parameter file, absolute path: /Users/stelios/workspace/stelios/pyradiomics/v2/params/params_1.yaml
Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 100, 'removeOutliers': None, 'resampledPixelSpacing': [2, 2, 2], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 5, 'voxelArrayShift': 300}
Enabled filters:
	 {'Original': {}, 'LoG': {'sigma': [2.0, 3.0, 4.0, 5.0]}, 'Wavelet': {}}
Enabled features:
	 {'shape': None, 'firstorder': None, 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'glrlm': None

### Setting up data and Feature Extraction

In [147]:
rootdir = dataset_folder/Path("dataset_to_process") # directory to store data we are interested in
features = [] # List of extracted features

for x in os.walk(rootdir):
    if 'BraTS' in x[0]:
        # sample_dir = /path/to/repo/v2/dataset/dataset_to_process/BraTS-MET-XXXXX-YYY
        sample_dir = x[0]

        # sample_file_base_name = BraTS-MET-XXXXX-YYY
        sample_file_base_name = sample_dir.split('/')[-1]

        # BraTS_MET_ID = XXXXX : int
        BraTS_MET_ID = int(sample_file_base_name.split('-')[-2])
        print(f"Fetching sample with ID: {BraTS_MET_ID}")

        # Get Image paths
        imagePath = os.path.join(sample_dir, sample_file_base_name+'-t2w.nii')
        maskPath = os.path.join(sample_dir, sample_file_base_name+'-seg.nii')

        # Feature extraction with pyradiomics
        print(f"Extracting features for sample: {sample_file_base_name}")
        results = extractor.execute(imagePath, maskPath)

        # Keep only specific features 
        temp = {k:v for k, v in results.items() if k.startswith(image_types)}
        temp.update({'BraTS_MET_ID':BraTS_MET_ID})
        features.append(temp)

Fetching sample with ID: 86
Extracting features for sample: BraTS-MET-00086-000
Fetching sample with ID: 284
Extracting features for sample: BraTS-MET-00284-000
Fetching sample with ID: 290
Extracting features for sample: BraTS-MET-00290-000
Fetching sample with ID: 247
Extracting features for sample: BraTS-MET-00247-000
Fetching sample with ID: 253
Extracting features for sample: BraTS-MET-00253-000
Fetching sample with ID: 119
Extracting features for sample: BraTS-MET-00119-000
Fetching sample with ID: 131
Extracting features for sample: BraTS-MET-00131-000
Fetching sample with ID: 125
Extracting features for sample: BraTS-MET-00125-000
Fetching sample with ID: 124
Extracting features for sample: BraTS-MET-00124-000
Fetching sample with ID: 130
Extracting features for sample: BraTS-MET-00130-000
Fetching sample with ID: 118
Extracting features for sample: BraTS-MET-00118-000
Fetching sample with ID: 252
Extracting features for sample: BraTS-MET-00252-000
Fetching sample with ID: 291


### Store to Dataframe and merge with label data

In [161]:
df = pd.DataFrame(features)
df = df.astype(float)
df = pd.merge(df, label, how='left', on='BraTS_MET_ID')
# feature_output_filename = dataset_folder/Path('features')/Path(params_file_name)
feature_output_filename = os.path.join(dataset_folder, "features", f'{params_file_name}.pkl')
pd.to_pickle(df, feature_output_filename)
print(f"Stored dataset {feature_output_filename} to pickle")

Stored dataset /Users/stelios/workspace/stelios/pyradiomics/v2/dataset/features/params_1.pkl to pickle


In [155]:
df

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,Death,Infratentorial,ET_num,Nec_num,Edema_num,ET_vol,Nec_vol,Edema_vol,Nec_ET_ratio,Edema_ET_ratio
0,0.743201,0.342911,14.042317,40.950293,34.92849839314596,40.24922359499622,34.92849839314596,42.37924020083418,3955.3333333333335,30.434283,...,0.0,1.0,3.0,0.0,3.0,0.458,0.000,3.469,0.000000,7.574236
1,0.786585,0.470726,39.115621,83.096308,76.02631123499285,86.55634003352961,94.76286192385707,95.5405672999695,109024.66666666667,65.362292,...,0.0,0.0,1.0,1.0,1.0,3.715,1.298,104.020,0.258927,20.750050
2,0.804832,0.676209,50.273607,74.346296,75.28612089887484,92.5418824100742,95.03683496413377,96.72641831474998,139839.0,59.836279,...,1.0,0.0,1.0,1.0,1.0,9.752,6.160,124.244,0.387129,7.808195
3,0.873655,0.642660,8.244739,12.829073,12.806248474865697,14.422205101855956,12.649110640673518,15.620499351813308,731.3333333333334,11.208178,...,1.0,1.0,1.0,1.0,1.0,0.345,0.071,0.425,0.170673,1.021635
4,0.755626,0.352583,39.270515,111.379385,104.6900186264192,112.44554237496477,105.84894897919393,119.43198901466893,119523.33333333333,84.161214,...,0.0,0.0,4.0,4.0,4.0,5.026,0.933,113.765,0.156570,19.091290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.666965,0.401032,58.450399,145.749926,128.06248474865697,147.95945390545344,154.983870128475,164.40194646049662,107098.66666666667,97.210169,...,1.0,1.0,47.0,2.0,30.0,9.563,0.152,99.068,0.015646,10.197427
196,0.775088,0.509471,21.146181,41.506154,38.41874542459709,45.65084884205331,42.190046219457976,48.78524367060187,17172.666666666668,32.170902,...,0.0,0.0,1.0,1.0,1.0,0.885,0.474,16.054,0.348786,11.813098
197,0.273180,0.166535,27.351589,164.239088,116.01724009818541,75.07329751649384,72.80109889280519,118.4736257569591,23359.666666666668,44.866798,...,1.0,0.0,3.0,2.0,3.0,1.598,0.130,21.638,0.075231,12.521991
198,0.752908,0.517420,38.664430,74.725478,76.94153624668537,88.81441324469807,74.96665925596525,88.90444308357148,102216.0,56.261406,...,0.0,0.0,1.0,1.0,1.0,2.147,0.467,99.474,0.178653,38.054323
